In [5]:
from blackscholes import Black76Call
from statistics import NormalDist
from scipy.stats import norm
import numpy as np
import pandas as pd
import mibian


In [6]:
def myround(x, base=10):
    return base * round(x/base) 

In [38]:
import numpy as np
from scipy.stats import norm
N = norm.cdf


def find_vol(target_value, S, K, T, r, *args):
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = Black76Call(S, K, T, r, sigma).price()
        vega = Black76Call(S, K, T, r, sigma).vega()
        diff = target_value - price  # our root
        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

def implied_volatility_bs(option_price, S, K,  T ,r, option_type='call'):
    tolerance = 0.0001
    lower_volatility = 0.01  # starting lower bound for volatility
    upper_volatility = 4.0   # starting upper bound for volatility

    while True:
        mid_volatility = (lower_volatility + upper_volatility) / 2
        theoretical_price = Black76Call(S , K, T, r, mid_volatility).price()
        
        if abs(theoretical_price - option_price) < tolerance:
            return mid_volatility
        
        if theoretical_price < option_price:
            lower_volatility = mid_volatility
        else:
            upper_volatility = mid_volatility

def strike_price_76(F ,r ,sigma ,  T , t ,delta, option_type = 'call' ):
    #  T option dte
    #  t time to future expiry
    if option_type == 'call':
        # result = (F* np.exp( - norm.ppf(delta * np.exp(r*t)) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2)) 
        result = (F* np.exp( - norm.ppf(delta ) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2))  # with less error
        return myround(result) 
    else:
        delta =  delta + 1
        # result = F* np.exp(  - norm.ppf(delta * np.exp(r*t)) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2) 
        result = F* np.exp(  - norm.ppf(delta ) * sigma * np.sqrt(T) -(sigma ** 2 * T) / 2) # with less error
        return myround(result)

In [8]:
def days_to_expiry(days):
    t = 80/365
    r = 0.055
    if days == 7:
        df = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="7")
        F = 5249.75
        sigma = 0.113388995411028
        T = 7/365
    elif days == 14:
        df = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="14")
        F = 5248.25
        sigma = 0.117092769160548
        T = 14/365
    elif days == 30:
        df = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="30")
        F = 5247.5
        sigma = 0.121898537553513
        T = 30/365
    elif days == 45:
        df = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="45")
        F = 5248.25
        sigma = 0.124105044640087
        T = 45/365
    elif days == 60:
        df = pd.read_excel(r"C:\Users\IBISP\Downloads\sp_chian.xlsx", sheet_name="60")
        F = 5247.75
        sigma = 0.124784832698593
        T = 60/365
    return df,F,r,t,T, sigma

In [220]:
df, F , r , t ,T, sigma  =  days_to_expiry(30)

In [223]:
Targeted_delta = 0.30

new_strike = strike_price_76(F ,sigma ,  T  ,Targeted_delta, option_type = 'call' )
option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
print(" new_strike , derived_delta , strike_volatility " , new_strike , derived_delta , strike_volatility)
i = 1
old_delta = derived_delta 
while i < 15:
    if abs(round(Targeted_delta,4) - round(derived_delta,4)) < 0.001:
        # if (Targeted_delta - old_delta) < (Targeted_delta - derived_delta):
        #     print(f"Strike price for ad {old_delta} delta:", old_strike)
        # else:
        if abs( round(Targeted_delta,4) - round(derived_delta,4)) < 0.0001:
            print(f"Strike price1 for {Targeted_delta} Targeted_delta:", new_strike)
        else:
            if Targeted_delta > 0.80:
                print(f"Strike price2 for {Targeted_delta} Targeted_delta:", new_strike - 10)
            elif Targeted_delta < 0.20:
                print(f"Strike price3 for {Targeted_delta} Targeted_delta:", new_strike + 10)
        # if abs( round(Targeted_delta,4) - round(derived_delta,4)) < 0.00001:
        #     if Targeted_delta > 80:
        #         print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike - 10)
        #     elif Targeted_delta < 20:
        #         print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike + 10)
        # if Targeted_delta > 0.9:
        #     if abs(Targeted_delta - derived_delta) < 0.001 :
        #         print(f"Strike price for {derived_delta} delta:", new_strike)
        #     else:
        #         print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike +10)
        # elif Targeted_delta < 0.4:
        #     if abs(Targeted_delta - derived_delta) < 0.001:
        #         print(f"Strike price for {derived_delta} delta:", new_strike)
        #     else:
        #         print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike -10)
        # else:
        #     print(f"Strike price for {derived_delta} delta:", new_strike)

        # print(i)
        break
    else:
        if Targeted_delta < derived_delta:

            old_strike = new_strike
            new_strike = old_strike + 10
            old_delta = derived_delta

            option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
            strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
            # derived_delta = mibian.BS([F, new_strike, r, T*365] , volatility= strike_volatility * 100)
            derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
            print(f"Actual Delta: {derived_delta}")
        else:

            old_strike = new_strike
            new_strike = old_strike - 10
            old_delta = derived_delta

            option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
            strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
            derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
        print(" Strike price4 = " , new_strike, "Delta = ", derived_delta)
    i += 1
if i == 15:
    if Targeted_delta > 0.7:
        print("Strike price5  = " , min(new_strike ,old_strike), "Delta = ", derived_delta)
    elif Targeted_delta < 0.3:
        print("Strike price6  = " , max(new_strike ,old_strike), "Delta = ", derived_delta)

 new_strike , derived_delta , strike_volatility  5340 0.30192846861820916 0.11470069242605944
Actual Delta: 0.28132530531689715
 Strike price4 =  5350 Delta =  0.28132530531689715
 Strike price4 =  5340 Delta =  0.30192846861820916
Actual Delta: 0.28132530531689715
 Strike price4 =  5350 Delta =  0.28132530531689715
 Strike price4 =  5340 Delta =  0.30192846861820916
Actual Delta: 0.28132530531689715
 Strike price4 =  5350 Delta =  0.28132530531689715
 Strike price4 =  5340 Delta =  0.30192846861820916
Actual Delta: 0.28132530531689715
 Strike price4 =  5350 Delta =  0.28132530531689715
 Strike price4 =  5340 Delta =  0.30192846861820916
Actual Delta: 0.28132530531689715
 Strike price4 =  5350 Delta =  0.28132530531689715
 Strike price4 =  5340 Delta =  0.30192846861820916
Actual Delta: 0.28132530531689715
 Strike price4 =  5350 Delta =  0.28132530531689715
 Strike price4 =  5340 Delta =  0.30192846861820916
Actual Delta: 0.28132530531689715
 Strike price4 =  5350 Delta =  0.2813253053

In [224]:
new_strike

5340

In [246]:
df, F , r , t ,T, sigma  =  days_to_expiry(30)

In [249]:
def strike_price_76(F  ,sigma ,  T  ,delta, option_type = 'call' ):
    #  T option dte
    #  t time to future expiry
    if option_type == 'call':
        # result = (F* np.exp( - norm.ppf(delta * np.exp(r*t)) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2)) 
        result = (F* np.exp( - norm.ppf(delta ) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2))  # with less error
        return myround(result) 
    else:
        delta =  delta + 1
        # result = F* np.exp(  - norm.ppf(delta * np.exp(r*t)) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2) 
        result = F* np.exp(  - norm.ppf(delta ) * sigma * np.sqrt(T) -(sigma ** 2 * T) / 2) # with less error
        return myround(result)
  



def first_strike(F ,r ,sigma ,  T  ,Targeted_delta ):
    new_strike = strike_price_76(F  ,sigma ,  T ,Targeted_delta, option_type = 'call' )

    # replace option price with fetching from new data
    option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2

    strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
    derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
    
    return new_strike , derived_delta , strike_volatility

def strike_for_target_delta(df , F ,r ,sigma ,  T , t ,Targeted_delta):
    new_strike , derived_delta , strike_volatility = first_strike(F ,r ,sigma ,  T  ,Targeted_delta)
    i = 1
    old_delta = derived_delta 
    while i < 15:
        if abs(round(Targeted_delta,4) - round(derived_delta,4)) < 0.001:
            
            if abs( round(Targeted_delta,4) - round(derived_delta,4)) < 0.0001:
                return new_strike , derived_delta
                # print(f"Strike price1 for {Targeted_delta} Targeted_delta:", new_strike)
            else:
                if Targeted_delta > 0.80:
                    return new_strike - 10 , derived_delta
                    # print(f"Strike price2 for {Targeted_delta} Targeted_delta:", new_strike - 10)
                elif Targeted_delta < 0.20:
                    return new_strike + 10 , derived_delta
                    # print(f"Strike price3 for {Targeted_delta} Targeted_delta:", new_strike + 10)
            
        else:
            if Targeted_delta < derived_delta:

                old_strike = new_strike
                new_strike = old_strike + 10
                old_delta = derived_delta

                option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
                strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
                # derived_delta = mibian.BS([F, new_strike, r, T*365] , volatility= strike_volatility * 100)
                derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
                # print(f"Actual Delta: {derived_delta}")

            else:

                old_strike = new_strike
                new_strike = old_strike - 10
                old_delta = derived_delta

                option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
                strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
                derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
            # print(" Strike price1 = " , new_strike, "Delta = ", derived_delta)
        i += 1  
    if i == 15:
        if Targeted_delta > 0.7:
            # print("Strike price  = " , min(new_strike ,old_strike), "Delta = ", derived_delta)
            return min(new_strike ,old_strike) , derived_delta
        elif Targeted_delta < 0.3:
            # print("Strike price  = " , max(new_strike ,old_strike), "Delta = ", derived_delta)
            return max(new_strike ,old_strike) , derived_delta
    return new_strike , derived_delta

In [250]:
df, F , r , t ,T, sigma  =  days_to_expiry(30)
Targeted_delta = 0.25

In [251]:
strike_for_target_delta(df , F ,r ,sigma ,  T , t ,Targeted_delta)

(5370, 0.24121046885236364)

In [212]:
# main code

# def strike_price_76(F  ,sigma ,  T  ,delta, option_type = 'call' ):
#     #  T option dte
#     #  t time to future expiry
#     if option_type == 'call':
#         # result = (F* np.exp( - norm.ppf(delta * np.exp(r*t)) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2)) 
#         result = (F* np.exp( - norm.ppf(delta ) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2))  # with less error
#         return myround(result) 
#     else:
#         delta =  delta + 1
#         # result = F* np.exp(  - norm.ppf(delta * np.exp(r*t)) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2) 
#         result = F* np.exp(  - norm.ppf(delta ) * sigma * np.sqrt(T) -(sigma ** 2 * T) / 2) # with less error
#         return myround(result)
    

# def first_strike(F ,r ,sigma ,  T  ,Targeted_delta ):
#     new_strike = strike_price_76(F  ,sigma ,  T ,Targeted_delta, option_type = 'call' )

#     # replace option price with fetching from new data
#     option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2

#     strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
#     derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
    
#     return new_strike , derived_delta , strike_volatility

# def strike_for_target_delta(df , F ,r ,sigma ,  T , t ,Targeted_delta):
#     new_strike , derived_delta , strike_volatility = first_strike(F ,r ,sigma ,  T  ,Targeted_delta)
#     i = 1
#     old_delta = derived_delta 
#     while i < 15:
#         if abs(round(Targeted_delta,4) - round(derived_delta,4)) < 0.001:
            
#             # if (Targeted_delta - old_delta) < (Targeted_delta - derived_delta):
#             #     print(f"Strike price for ad {old_delta} delta:", old_strike)
#             # else:
#             if abs( round(Targeted_delta,4) - round(derived_delta,4)) < 0.00000001:
#                 # print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike)
#                 return new_strike , derived_delta
#             if abs( round(Targeted_delta,4) - round(derived_delta,4)) < 0.00001:
#                 if Targeted_delta > 80:
#                     # print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike - 10)
#                     return new_strike - 10 , derived_delta
#                 elif Targeted_delta < 20:
#                     # print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike + 10)
#                     return new_strike + 10 , derived_delta
#             break
#         else:
#             if Targeted_delta < derived_delta:

#                 old_strike = new_strike
#                 new_strike = old_strike + 10
#                 old_delta = derived_delta

#                 option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
#                 strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
#                 # derived_delta = mibian.BS([F, new_strike, r, T*365] , volatility= strike_volatility * 100)
#                 derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
#                 # print(f"Actual Delta: {derived_delta}")

#             else:

#                 old_strike = new_strike
#                 new_strike = old_strike - 10
#                 old_delta = derived_delta

#                 option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
#                 strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
#                 derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
#             # print(" Strike price1 = " , new_strike, "Delta = ", derived_delta)
#         i += 1  
#     if i == 15:
#         if Targeted_delta > 0.6:
#             # print("Strike price  = " , min(new_strike ,old_strike), "Delta = ", derived_delta)
#             return min(new_strike ,old_strike) , derived_delta
#         elif Targeted_delta < 0.4:
#             # print("Strike price  = " , max(new_strike ,old_strike), "Delta = ", derived_delta)
#             return max(new_strike ,old_strike) , derived_delta